# <font color=darkblue> Machine Learning model deployment with Flask framework</font>

## <font color=Blue>Used Cars Price Prediction Application</font>

### Objective:
1. To build a Machine learning regression model to predict the selling price of the used cars based on the different input features like fuel_type, kms_driven, type of transmission etc.
2. Deploy the machine learning model with the help of the flask framework.

### Dataset Information:
#### Dataset Source: https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho?select=CAR+DETAILS+FROM+CAR+DEKHO.csv
This dataset contains information about used cars listed on www.cardekho.com
- **Car_Name**: Name of the car
- **Year**: Year of Purchase
- **Selling Price (target)**: Selling price of the car in lakhs
- **Present Price**: Present price of the car in lakhs
- **Kms_Driven**: kilometers driven
- **Fuel_Type**: Petrol/diesel/CNG
- **Seller_Type**: Dealer or Indiviual
- **Transmission**: Manual or Automatic
- **Owner**: first, second or third owner


### 1. Import required libraries

In [85]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

### 2. Load the dataset

In [86]:
df = pd.read_csv("download.csv")
# first few samples
df.head(5)

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


### 3. Check the shape and basic information of the dataset.

In [87]:
# Shape (rows, columns)
df.shape

(301, 9)

In [88]:
# info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       301 non-null    object 
 1   Year           301 non-null    int64  
 2   Selling_Price  301 non-null    float64
 3   Present_Price  301 non-null    float64
 4   Kms_Driven     301 non-null    int64  
 5   Fuel_Type      301 non-null    object 
 6   Seller_Type    301 non-null    object 
 7   Transmission   301 non-null    object 
 8   Owner          301 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 21.3+ KB


In [89]:
# Missing values
df.isnull().sum()/len(df)*100

Car_Name         0.0
Year             0.0
Selling_Price    0.0
Present_Price    0.0
Kms_Driven       0.0
Fuel_Type        0.0
Seller_Type      0.0
Transmission     0.0
Owner            0.0
dtype: float64

No missing values

### 4. Check for the presence of the duplicate records in the dataset? If present drop them

In [90]:
# duplicates
len(df[df.duplicated()])

2

In [91]:
#dropping 2 duplicates
df.drop_duplicates(inplace=True)

In [92]:
# duplicates after dropping
len(df[df.duplicated()])

0

In [93]:
df.shape

(299, 9)

### 5. Drop the columns which you think redundant for the analysis.

In [94]:
# first lets see all the columns
df.columns

Index(['Car_Name', 'Year', 'Selling_Price', 'Present_Price', 'Kms_Driven',
       'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner'],
      dtype='object')

In [95]:
# Dropping redundant columns
df = df.drop(columns = ['Car_Name'], axis = 1)

In [96]:
# after dropping new shape of data
df.shape

(299, 8)

### 6. Extract a new feature called 'age_of_the_car' from the feature 'year' and drop the feature year

In [97]:
df['age_of_the_car'] = 2024 - df['Year']

In [98]:
df.drop('Year', axis = 1, inplace =True)

In [99]:
df.head(5)

,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,age_of_the_car
0,3.35,5.59,27000,Petrol,Dealer,Manual,0,10
1,4.75,9.54,43000,Diesel,Dealer,Manual,0,11
2,7.25,9.85,6900,Petrol,Dealer,Manual,0,7
3,2.85,4.15,5200,Petrol,Dealer,Manual,0,13
4,4.60,6.87,42450,Diesel,Dealer,Manual,0,10


### 7. Encode the categorical columns

In [100]:
df['Fuel_Type'].unique()

array(['Petrol', 'Diesel', 'CNG'], dtype=object)

In [101]:
df['Seller_Type'].unique()

array(['Dealer', 'Individual'], dtype=object)

In [102]:
df['Transmission'].unique()

array(['Manual', 'Automatic'], dtype=object)

In [103]:
# Manual encoding
df['Fuel_Type'] = df['Fuel_Type'].replace({'Petrol': 0, 'Diesel': 1, 'CNG': 2})

df['Seller_Type'] = df['Seller_Type'].replace({'Dealer': 0, 'Individual': 1})

df['Transmission'] = df['Transmission'].replace({'Manual': 0, 'Automatic': 1})

In [104]:
df['Fuel_Type'].unique()

array([0, 1, 2])

In [105]:
df['Seller_Type'].unique()

array([0, 1])

In [106]:
df['Transmission'].unique()

array([0, 1])

In [107]:
df.head(5)

,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,age_of_the_car
0,3.35,5.59,27000,0,0,0,0,10
1,4.75,9.54,43000,1,0,0,0,11
2,7.25,9.85,6900,0,0,0,0,7
3,2.85,4.15,5200,0,0,0,0,13
4,4.60,6.87,42450,1,0,0,0,10


### 8. Separate the target and independent features.

In [108]:
X= df.drop('Selling_Price', axis=1)
y = df['Selling_Price']

### 9. Split the data into train and test.

In [109]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=0)
print(X_train.shape, X_test.shape)                
print(y_train.shape, y_test.shape)

(209, 7) (90, 7)
(209,) (90,)


### 10. Build a Random forest Regressor model and check the r2-score for train and test.

In [110]:
rf =RandomForestRegressor()
rf.fit(X_train,y_train)

RandomForestRegressor()

In [111]:
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

r2_train= r2_score(y_train, y_train_pred)
r2_test= r2_score(y_test, y_test_pred)

print(r2_train)
print(r2_test)

0.981590346557568
0.9044366901512306


### 11. Create a pickle file with an extension as .pkl

In [112]:
import pickle
pickle.dump(rf, open('model.pkl','wb'))

### 12. Create new folder/new project in visual studio/pycharm that should contain the "model.pkl" file *make sure you are using a virutal environment and install required packages.*

### a) Create a basic HTML form for the frontend

Create a file **index.html** in the templates folder and copy the following code.

#Since its giving the following error
#<!DOCTYPE html>
#SyntaxError: invalid syntax
#We used Markdown cell to write our HTML
    
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Document</title>
</head>

<body>

    <div class="hero-image">
      <div class="hero-text">

        <h1 style="font-size:50px">Used Car Price Predictor</h1>
         <br><br><h3>{{ prediction_text }}<h3>
      </div>
    </div>

     <style>

        body, html {
          height: 100%;
          margin: 0;
          font-family: Arial, Helvetica, sans-serif;
        }

        .hero-image {
          background-image: linear-gradient(rgba(0, 0, 0, 0.5), rgba(0, 0, 0, 0.5)), url('/static/image.jpg');
          height: 50%;
          background-position: bottom;
          background-repeat: no-repeat;
          background-size: cover;
          position: relative;
        }

        .hero-text {
          text-align: center;
          position: absolute;
          top: 50%;
          left: 50%;
          transform: translate(-50%, -50%);
          color: white;
        }

    </style>


    <div style="color:	rgb(0, 0, 0)">
        <form action="{{ url_for('predict')}}" method="post">
            <h2>Enter Car Details: </h2>
            <h3>Age of the car(In years)</h3>
            <input id="first" name="Age_of_the_car" type="number ">
            <h3>Present Showroom Price(In lakhs)</h3><br><input id="second" name="Present_Price" required="required">
            <h3>Kilometers Driven</h3><input id="third" name="Kms_Driven" required="required">
            <h3>Owner Type (0/1/3)</h3><br><input id="fourth" name="Owner" required="required">
            <h3>Fuel type</h3><br><select name="Fuel_Type" id="fuel" required="required">
                <option value="0">Petrol</option>
                <option value="1">Diesel</option>
                <option value="2">CNG</option>
            </select>
            <h3>Seller Type</h3><br><select name="Seller_Type" id="resea" required="required">
                <option value="0">Dealer</option>
                <option value="1">Individual</option>
            </select>
            <h3>Transmission type</h3><br><select name="Transmission" id="research" required="required">
                <option value="0">Manual Car</option>
                <option value="1">Automatic Car</option>
            </select>
            <br><br><button id="sub" type="submit ">Predict Selling Price</button>
            <br>


        </form>

    </div>

    <style>
	body {
            background-color: 101, 10, 20;
            text-align: center;
            padding: 0px;
	    font-family: Helvetica;
        }

        #research {
            font-size: 18px;
            width: 200px;
            height: 23px;
            top: 23px;
        }

        #box {
            border-radius: 60px;
            border-color: 45px;
            border-style: solid;
            text-align: center;
            background-color: white;
            font-size: medium;
            position: absolute;
            width: 700px;
            bottom: 9%;
            height: 850px;
            right: 30%;
            padding: 0px;
            margin: 0px;
            font-size: 14px;
        }

        #fuel {
            width: 83px;
            height: 43px;
            text-align: center;
            border-radius: 14px;
            font-size: 20px;
        }

        #fuel:hover {
            background-color: white;
        }

        #research {
            width: 150px;
            height: 43px;
            text-align: center;
            border-radius: 14px;
            font-size: 18px;
        }

        #research:hover {
            background-color: white;
        }

        #resea {
            width: 99px;
            height: 43px;
            text-align: center;
            border-radius: 14px;
            font-size: 18px;
        }

        #resea:hover {
            background-color: white;
        }

        #sub {
            background-color: Green;
            font-family:'Helvetica' monospace;
            font-weight: bold;
            width: 180px;
            height: 60px;
            text-align: center;
            border-radius: 20px;
            font-size: 18px;
            color: white;
        }

        #sub:hover {
            background-color: white;
        }

        #first {
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }

        #second {
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }

        #third {
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }

        #fourth {
            border-radius: 14px;
            height: 25px;
            font-size: 20px;
            text-align: center;
        }
    </style>
</body>

</html>

### b) Create app.py file and write the predict function

In [113]:
from flask import Flask, render_template, request, jsonify
import pickle
import numpy as np
import sklearn

#read pickle
app= Flask(__name__)
model= pickle.load(open('model.pkl','rb'))


@app.route('/', methods =['GET'])
def Home ():
    return render_template('index.html')

@app.route('/predict', methods =['POST'])
def predict ():
    if request.method == 'POST':
        Present_Price = float(request.form['Present_Price'])
        Kms_Driven = int(request.form['Kms_Driven'])
        Owner = int(request.form['Owner'])
        Fuel_type = request.form['Fuel_Type']
        Age_of_the_car= request.form['Age_of_the_car']
        Seller_Type = request.form['Seller_Type']
        Transmission =request.form['Transmission']
        
        prediction= model.predict([[Present_Price, Kms_Driven, Owner, Age_of_the_car, Fuel_Type, Seller_Type,Transmission]])
        output = round (prediction[0],2)
        return render_template('index.html', prediction_text = "You can sell your car at {} lakhs".format(output))     
    
    if __name__ == '__main__':
        app.run(debug=True)

### 13. Run the app.py python file which will render to index html page then enter the input values and get the prediction.

Ran the app.py file, which was rendering to index.html (stored in templates folder)

## OBSERVATIONS: 
Just for instance,
1. I have selected car- Ritz from the 1st row
2. Readings were entered according to the data from that row
3. In the data, the selling price mentioned was= 3.35 lakhs
4. The amount predicted by "Used Car Price Predictor" is = 3.56 lakhs
5. The predicted value is very close to the mentioned selling price. 
6. I have entered couple of random values and the prices predicted were good and sensible.
#### This implies that the Predictor is effective in estimating the selling price of used cars.

### Happy Learning :)